In [1]:
import datetime as dt
import pandas as pd

In [2]:
df = pd.read_parquet("/home/donsson/proyectos/API/ventashistoricas105semanas.parquet")

In [3]:
df.sample(5)

,origin,product_id,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,sales_team_name,partner_id_num,client_name_inv
147843,False,"[18922, '[BLS00548125] GS548 FILTRO ACEITE INT...",156600.0,1457081,1.0,[BLS00548125] GS548 FILTRO ACEITE INTERNATIONA...,PV1E9205 Mostrador Cota T2/2697,2025-01-24 17:14:34,PV1E9205,396260,PRINCIPAL COTA,YURI NATALIA GAONA,SUCURSALES / Ventas Mostrador Cota,28483,[CO9014513807] LUBRITROCAS TRUCKS MC SAS
120811,False,"[19416, '[BCE00443125] G443 FILTRO COMBUSTIBLE...",13900.0,1488252,1.0,[BCE00443125] G443 FILTRO COMBUSTIBLE UNIVERSA...,PV6E16313 Mostrador Barranquilla T2/22208,2025-03-11 18:59:35,PV6E16313,404824,SUCURSAL BARRANQUILLA,PAMELA TRUJILLO,SUCURSALES / BARRANQUILLA / Ventas mostrador B...,36133,[CO8000437691] SERVICIOS DE DRAGADOS Y CONSTRU...
70180,False,"[17686, '[DAB04788025] DA4788 FILTRO AIRE 2o. ...",37100.0,1548967,1.0,[DAB04788025] DA4788 FILTRO AIRE 2o. JAC 1083 ...,PV3E18612 Mostrador Valladolid T2/14519,2025-06-10 17:15:14,PV3E18612,420654,SUCURSAL VALLADOLID,SEBAS RODRIGUEZ,SUCURSALES / Ventas Mostrador Valladolid,34017,[CC79888951] JUAN PABLO MELO OSPINA
277926,BD1\OUT\53951,"[25669, '[DAB08168025] DA8168 FILTRO AIRE DONS...",57900.0,1300388,1.0,[DAB08168025] DA8168 FILTRO AIRE DONSSON - FOT...,FCOT19052,2024-05-06 16:01:30,FCOT19052,354400,PRINCIPAL COTA,LUIS EDUARDO ARANGO,EXTERNOS,37358,[CO9007029864] OPERLOG COLOMBIA SAS
159841,False,"[23807, '[DCS30536137] GS536A30 FILTRO SEP AGU...",66600.0,1442627,1.0,[DCS30536137] GS536A30 FILTRO SEP AGUA/COMB CH...,PV2E36118 Mostrador Calle 6 T2/19274,2024-12-28 14:47:27,PV2E36118,392596,SUCURSAL CALLE 6,JAIME ANDRES CABALLERO CABALLERO,SUCURSALES / Ventas Mostrador Cll6,34672,[CC3104259] MILTON ELIN PENAGOS MENDEZ


In [4]:
import re
import unicodedata



mapa_sucursales = {
    'BD1': 'PRINCIPAL COTA',
    'BD2': 'SUCURSAL CALLE 6',
    'BD3': 'SUCURSAL VALLADOLID',
    'BD4': 'SUCURSAL NORTE',
    'BD5': 'SUCURSAL MEDELLIN',
    'BD6': 'SUCURSAL BARRANQUILLA',
    'BD7':' SUCURSAL BUCARAGAMNGA ',
    'BD8':'MOSTRADOR COTA',
    'BD9':'SUCURSAL CALI',
    'BD11': 'CUMMINS DE LOS ANDES',
    'BD12': 'CUMMINS DE LOS ANDES MEDELLIN',
    'BD13': 'CSS CONSTRUCTORES',
    'BD14': 'ANTIOQUEÑA DE LUBRICANTES SGP SAS',
    'BD15': 'ASEO CAPITAL',
    'BD16': 'COVIANDES',
    'BD18': 'CEMEX BOSA',
    'BD19': 'Cemex ibague (eliminar)',
    'BD21': 'CEMEX CENTENARIO',
    'BD22': 'PENDIENTE POR FACTURAR CIERRE CONT.',
    'BD24': 'FILTRO EN REPROCESO',
    'BD26': 'PRESTAMOS INTERNOS',
    'BD31': 'EXP.CTP',
    'BD32': 'EXP. ECUADOR',
    'BD34':'EXP.DONSSON.USA',
    'BD40': 'DESCUADRE B1',
    'BD MP':'MATERIA PRIMA',
    'BD41':'BODEGA VENDEDOR WILMER GIL',
    'BD33':'EXP.REP.DOMINICANA',
    'BD42':'STOCK SIGMA ENERGY'
    # Agrega más códigos si los encuentras en tus datos, siguiendo este formato.
    }




def clasificar_sucursal(invoice_name):
    if pd.isna(invoice_name):
        return "VENDEDOR EXTERNO"
    if "Mostrador" in invoice_name:
        return "MOSTRADOR"
    return "VENDEDOR EXTERNO"
    
df["Sucursal_tipo"] = df["invoice_name"].apply(clasificar_sucursal)


df["client_name"] = df["client_name_inv"].str.split("] ", expand=True)[1].str.strip()



#Quitar las facturas que salen como SO son no vendibles
df = df[~(df["origin"].str.contains("SO"))] 


# Normalizar los separadores para que todos sean iguales
df["origin_norm"] = df["origin"].str.replace(r"[\\/]", "/", regex=True)

# Extraer solo el BD y el número (ejemplo: BD11, BD9, etc.)
df["origen"] = df["origin_norm"].str.extract(r"^(BD\d+)")

df["Bodega"] = df["origen"].map(mapa_sucursales)
df["Bodega"] = df["Bodega"].fillna(df["store_name"])

df = df.drop(columns=["origin_norm"])

df = df.drop(columns=["origin"])

df["product_ref"] = df["product_name"].str.extract(r"\[([A-Z0-9]+)\]")

df["date_invoice"] = pd.to_datetime(df["date_invoice"])
df["mes"] = df["date_invoice"].dt.month
df["año"] = df["date_invoice"].dt.year
df["dia"] = df["date_invoice"].dt.day

df = df[~(df["product_ref"].isna())]

pd.set_option('display.max_columns', None)

df['marca'] = df['product_ref'].apply(
    lambda x: 'BALDWIN' if str(x).endswith('125') else
              'DONSSON' if str(x).endswith('025') else
              'AUT*PARTS' if str(x).endswith('189') else
              'RACOR BRASIL' if str(x).endswith('137') else
              'RACOR USA' if str(x).endswith('138') else
              'OTRA'
)

In [5]:
from datetime import datetime

# Fecha y hora actual en formato YYYYMMDD_HHMMSS
timestamp = datetime.now().strftime("%Y%m%d")

# Nombre dinámico
filename = f"/home/donsson/proyectos/VENTAS/prom_ventas/promedio_ventas_{timestamp}.xlsx"

# Guardar el Excel
df.to_excel(filename, index=False)

print(f"Archivo guardado: {filename}")

#Tiempo promedio 3 minutos

Archivo guardado: /home/donsson/proyectos/VENTAS/prom_ventas/promedio_ventas_20251014.xlsx
